In [1]:
import pandas as pd
import numpy as np

In [2]:
df = pd.read_csv('data/events.csv', low_memory=False)

In [3]:
df['timestamp'] = pd.to_datetime(df['timestamp'])
df.sort_values(by='timestamp', inplace=True)

In [4]:
eventos = df.loc[df['event'].isin(['conversion', 'checkout']),\
                 ['timestamp', 'event', 'person', 'sku', 'model'\
                  ,'condition', 'storage', 'color']].sort_values(by='timestamp')

In [5]:
eventos = eventos.loc[((eventos['event'] + eventos['event'].shift(1) == 'conversioncheckout') &\
                                    (eventos['person'] == eventos['person'].shift(1))) |\
                                    ((eventos['event'] + eventos['event'].shift(-1) == 'checkoutconversion') &\
                                    (eventos['person'] == eventos['person'].shift(-1)))]
eventos['dif_time'] = eventos.groupby('person')['timestamp'].diff() / np.timedelta64(1, 'h')

In [6]:
eventos = eventos.loc[((eventos['event'] == 'checkout') & (eventos['dif_time'].shift(-1) < 1.0)) |\
                     ((eventos['event'] == 'conversion') & (eventos['dif_time'] < 1.0))]

In [7]:
personas_que_compraron_on_site = set(eventos['person'].unique())

In [8]:
users = pd.read_csv('data/features_basicas.csv', low_memory=False)
users['compro_onsite'] = users['person'].isin(personas_que_compraron_on_site)
users.set_index('person', inplace=True)
users = users[['compro_onsite']]

In [9]:
users.to_csv('data/feature_compra_onsite.csv')